# Parallelizing Remote QNode Execution

This notebook demonstrates the importance of parallelizing the evaluation of cost functions gradients when using remote hardware.
For this demonstration, we use open-access IBM quantum hardware simulators and qNetVO's parallelization functionality built on dask.
These simulators have relatively short queue-times in comparison to the available quantum computers yet reveal the advantages of parallelizing the web requests used to invoke the remote IBM Q services.

In [1]:
import pennylane as qml
from pennylane import numpy as np

import qnetvo as qnet

In [2]:
from qiskit import IBMQ

# For details regarding integration between PennyLane and IMB Quantum,
# see https://pennylaneqiskit.readthedocs.io/en/latest/devices/ibmq.html#accounts-and-tokens
provider = IBMQ.load_account()

## Setup

For simplicity, we consider a CHSH scenario ansatz with a static Bell state preparation and local qubit measurements optimized over the $xz$-plane.
The qnodes are executed/trained remotely on the `ibmq_qasm_simulator`.
In certain cases, qnode execution is performed locally for greater efficiency.

In [3]:
prep_nodes = [
    qnet.PrepareNode(1, [0,1], qnet.ghz_state, 0)
]
meas_nodes = [
    qnet.MeasureNode(2, 2, [0], qnet.local_RY, 1),
    qnet.MeasureNode(2, 2, [1], qnet.local_RY, 1)
]

dev_ibm_qasm = {
    "name" : "qiskit.ibmq",
    "shots" : 2000,
    "backend" : "ibmq_qasm_simulator",
    "provider" : provider
}

local_sim_chsh_ansatz = qnet.NetworkAnsatz(prep_nodes, meas_nodes)
ibm_sim_chsh_ansatz = qnet.NetworkAnsatz(
    prep_nodes, meas_nodes, dev_kwargs = dev_ibm_qasm
)

## QNode Execution Parallelization

We now demonstrate the performance gains granted by parallelizing qnode execution across IBM Q hardware simulator devices.
In this example the CHSH cost function requires 4 remote qnode executions which are first run serially and then, in parallel.

### Non-Parallelized Remote QNode Execution

By default, PennyLane will chain web requests invoking remote qnode execution serially. 
This is a great ineffeciency granted that each of these qnode executions are independent from each other.

In [4]:
%%time

chsh_cost = qnet.chsh_inequality_cost_fn(ibm_sim_chsh_ansatz)

np.random.seed(13)
rand_settings = ibm_sim_chsh_ansatz.rand_network_settings()

chsh_cost(*rand_settings)

CPU times: user 468 ms, sys: 64 ms, total: 532 ms
Wall time: 40.2 s


tensor(-0.58, requires_grad=True)

### Parallelized Remote QNode Execution

A factor of roughly 2-4x speedup is found by parallelizing the qnode execution across 4 separate threads.
This speedup persists even when a single remote device runs all qnode executions because the parallelized web requests populate the queue sooner than serial web requests.



In [5]:
%%time

parallel_chsh_cost = qnet.chsh_inequality_cost_fn(
    ibm_sim_chsh_ansatz, parallel=True
)

np.random.seed(13)
rand_settings = ibm_sim_chsh_ansatz.rand_network_settings()

parallel_chsh_cost(*rand_settings)

CPU times: user 447 ms, sys: 50.8 ms, total: 498 ms
Wall time: 8.52 s


tensor(-0.555, requires_grad=True)

## QNode Gradients on Remote Hardware

In this section we demonstrate that performance gains granted by parallelizing gradient computation of remote qnodes.
The gradient is evaluated using the parameter shift rule.


### Non-Parallelized Remote Gradient Computation

By default, all cost functions and gradients are evaluated serially on remote hardware.
This requires 28 web requests in total to IBM remote simulator.

In [6]:
%%time

ibm_sim_chsh_cost = qnet.chsh_inequality_cost_fn(ibm_sim_chsh_ansatz)

np.random.seed(13)
rand_settings = ibm_sim_chsh_ansatz.rand_network_settings()

qnet.gradient_descent(
    ibm_sim_chsh_cost,
    rand_settings,
    num_steps=1,
    sample_width=1,
    step_size=0.1
)

iteration :  0 , score :  0.6610000000000001


elapsed time :  153.12486505508423


CPU times: user 1.38 s, sys: 117 ms, total: 1.5 s
Wall time: 3min 32s


{'datetime': '2022-10-31T17:54:32Z',
 'opt_score': tensor(1.147, requires_grad=True),
 'opt_settings': [tensor(1.86790571, requires_grad=True),
  tensor(-1.60117715, requires_grad=True),
  tensor(2.06030211, requires_grad=True),
  tensor(2.73428852, requires_grad=True)],
 'scores': [tensor(0.661, requires_grad=True),
  tensor(1.147, requires_grad=True)],
 'samples': [0, 1],
 'settings_history': [[tensor(1.74485571, requires_grad=True),
   tensor(-1.64907715, requires_grad=True),
   tensor(2.03750211, requires_grad=True),
   tensor(2.92638852, requires_grad=True)],
  [tensor(1.86790571, requires_grad=True),
   tensor(-1.60117715, requires_grad=True),
   tensor(2.06030211, requires_grad=True),
   tensor(2.73428852, requires_grad=True)]],
 'step_times': [153.12486505508423, 153.12486505508423],
 'step_size': 0.1}

### Parallelized Remote Gradient Computation

To speed up the gradient computation, we parallelize the parameter shift rule across 4 web requests.
In total only 20 web requests to remote IBM hardware are needed where we are able to drop 8 requests from the previous example by evaluating the cost function locally.
The training is then isolated in 20 web requests split across four independent threads.
We see a rough 4x improvement in training time.

In [7]:
%%time

local_sim_chsh_cost = qnet.chsh_inequality_cost_fn(local_sim_chsh_ansatz)
parallel_grad_fn = qnet.parallel_chsh_grad_fn(
    ibm_sim_chsh_ansatz, diff_method="parameter-shift"
)


np.random.seed(13)
rand_settings = ibm_sim_chsh_ansatz.rand_network_settings()

qnet.gradient_descent(
    local_sim_chsh_cost,
    rand_settings,
    num_steps=1,
    sample_width=1,
    step_size=0.1,
    grad_fn=parallel_grad_fn
)

iteration :  0 , score :  0.6183525458603143


elapsed time :  67.95212507247925
CPU times: user 896 ms, sys: 77.4 ms, total: 973 ms
Wall time: 1min 8s


{'datetime': '2022-10-31T17:58:05Z',
 'opt_score': tensor(1.1252902, requires_grad=True),
 'opt_settings': [tensor(1.86495571, requires_grad=True),
  tensor(-1.60022715, requires_grad=True),
  tensor(2.06025211, requires_grad=True),
  tensor(2.73523852, requires_grad=True)],
 'scores': [tensor(0.61835255, requires_grad=True),
  tensor(1.1252902, requires_grad=True)],
 'samples': [0, 1],
 'settings_history': [[tensor(1.74485571, requires_grad=True),
   tensor(-1.64907715, requires_grad=True),
   tensor(2.03750211, requires_grad=True),
   tensor(2.92638852, requires_grad=True)],
  [tensor(1.86495571, requires_grad=True),
   tensor(-1.60022715, requires_grad=True),
   tensor(2.06025211, requires_grad=True),
   tensor(2.73523852, requires_grad=True)]],
 'step_times': [67.95212507247925, 67.95212507247925],
 'step_size': 0.1}

In [8]:
%%time

local_sim_chsh_cost = qnet.chsh_inequality_cost_fn(local_sim_chsh_ansatz)
parallel_grad_fn = qnet.parallel_chsh_grad_fn(
    ibm_sim_chsh_ansatz, natural_grad=True, diff_method="parameter-shift"
)


np.random.seed(13)
rand_settings = ibm_sim_chsh_ansatz.rand_network_settings()

qnet.gradient_descent(
    local_sim_chsh_cost,
    rand_settings,
    num_steps=1,
    sample_width=1,
    step_size=0.1,
    grad_fn=parallel_grad_fn
)

iteration :  0 , score :  0.6183525458603143


elapsed time :  184.7216079235077
CPU times: user 1.27 s, sys: 96.9 ms, total: 1.36 s
Wall time: 3min 4s


{'datetime': '2022-10-31T17:59:13Z',
 'opt_score': tensor(1.542151, requires_grad=True),
 'opt_settings': [tensor(1.99008941, requires_grad=True),
  tensor(-1.55130986, requires_grad=True),
  tensor(2.07730249, requires_grad=True),
  tensor(2.54358715, requires_grad=True)],
 'scores': [tensor(0.61835255, requires_grad=True),
  tensor(1.542151, requires_grad=True)],
 'samples': [0, 1],
 'settings_history': [[tensor(1.74485571, requires_grad=True),
   tensor(-1.64907715, requires_grad=True),
   tensor(2.03750211, requires_grad=True),
   tensor(2.92638852, requires_grad=True)],
  [tensor(1.99008941, requires_grad=True),
   tensor(-1.55130986, requires_grad=True),
   tensor(2.07730249, requires_grad=True),
   tensor(2.54358715, requires_grad=True)]],
 'step_times': [184.7216079235077, 184.7216079235077],
 'step_size': 0.1}